# Load NLP datasets

In [1]:
# datasets 라이브러리 설치 (Hugging Face의 데이터셋 로딩 도구)
!pip install datasets

In [2]:
from datasets import load_dataset

# Hugging Face > Blpeng/nsmc (네이버 영화 감성 분석) 데이터셋 로드
dataset = load_dataset("Blpeng/nsmc")

# train 150,000 / test 50,000
print(dataset)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/34.0 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


ratings_train.csv:   0%|          | 0.00/15.7M [00:00<?, ?B/s]

ratings_test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['Unnamed: 0', 'id', 'document', 'label'],
        num_rows: 50000
    })
})


In [3]:
dataset['train'][0]

{'Unnamed: 0': 0, 'id': 9976970, 'document': '아 더빙.. 진짜 짜증나네요 목소리', 'label': 0}

# Load Pretrained Word Vectors

In [4]:
# GloVe 임베딩 파일(6B, 300차원) 다운로드 및 압축 해제
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2025-10-29 16:03:33--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-10-29 16:03:34--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  4.99MB/s    in 2m 39s  

2025-10-29 16:06:13 (5.16 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflatin

In [5]:
!wget https://nlp.stanford.edu/data/glove.42B.300d.zip

--2025-10-29 16:06:41--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2025-10-29 16:06:41--  https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Saving to: ‘glove.42B.300d.zip’

glove.42B.300d.zip  100%[===================>]   1.75G  4.96MB/s    in 5m 53s  

2025-10-29 16:12:34 (5.07 MB/s) - ‘glove.42B.300d.zip’ saved [1877800501/1877800501]



# Pre-processing

In [6]:
# ### 데이터 확인용
with open("./glove.6B.300d.txt", 'r') as f:
    for idx, line in enumerate(f):
        if idx < 5:
            line = line.strip().split(' ')
            word, vector = line[0], line[1:]
            print(idx, " | ", word, " | ", vector)

0  |  the  |  ['0.04656', '0.21318', '-0.0074364', '-0.45854', '-0.035639', '0.23643', '-0.28836', '0.21521', '-0.13486', '-1.6413', '-0.26091', '0.032434', '0.056621', '-0.043296', '-0.021672', '0.22476', '-0.075129', '-0.067018', '-0.14247', '0.038825', '-0.18951', '0.29977', '0.39305', '0.17887', '-0.17343', '-0.21178', '0.23617', '-0.063681', '-0.42318', '-0.11661', '0.093754', '0.17296', '-0.33073', '0.49112', '-0.68995', '-0.092462', '0.24742', '-0.17991', '0.097908', '0.083118', '0.15299', '-0.27276', '-0.038934', '0.54453', '0.53737', '0.29105', '-0.0073514', '0.04788', '-0.4076', '-0.026759', '0.17919', '0.010977', '-0.10963', '-0.26395', '0.07399', '0.26236', '-0.1508', '0.34623', '0.25758', '0.11971', '-0.037135', '-0.071593', '0.43898', '-0.040764', '0.016425', '-0.4464', '0.17197', '0.046246', '0.058639', '0.041499', '0.53948', '0.52495', '0.11361', '-0.048315', '-0.36385', '0.18704', '0.092761', '-0.11129', '-0.42085', '0.13992', '-0.39338', '-0.067945', '0.12188', '0.167

In [7]:
# key: 단어, value: GloVe 임베딩 벡터인 딕셔너리
word_emb = {}
# 각 단어에 고유한 인덱스를 부여하기 위한 딕셔너리
  # '<PAD>'와 '<OOV>'는 특수 토큰으로 미리 정의하는 것이 일반적
word_to_index = {'<PAD>': 0, '<OOV>': 1}

# 임베딩 파일 읽기 모드로 열기
with open("./glove.6B.300d.txt", 'r') as f:
    for line in f:
        line = line.strip().split(' ')                # 양쪽 공백을 제거하고 공백을 기준으로 단어와 벡터를 분리
        word, vector = line[0], line[1:]
        word_emb[line[0]] = list(map(float, vector))  # word_emb에 단어와 해당 벡터(실수형으로 변환)를 저장
        word_to_index[word] = len(word_to_index)      # word_to_index에 단어를 추가

In [8]:
# ### 데이터 확인용
print(len(word_emb))
print(len(word_to_index))

400000
400002


In [9]:
# 전처리 단계 : 감성 분석 모델 학습을 위해 텍스트 데이터를 숫자로 변환(임베딩)하고 데이터 길이를 통일

# 텍스트 클리닝을 위한 정규표현식을 처리
import re
# 배열 생성과 같은 수치 연산에 사용
import numpy as np

# 입력 시퀀스의 최대 길이 설정 (단어 개수 기준)
# 시퀀스가 이 길이를 초과하면 잘리고(Truncation), 부족하면 패딩을 추가해 고정 길이로 맞춤
max_seq_len = 128

# 1. 텍스트 정제
def text_cleaning(text):

    # 특수 문자 제거, 불용어 처리, 형태소 분석
    return text

# 2. 텍스트 데이터로 사전(Vocabulary) 구축, 정수 인덱스 배열로 변환, 문장 길이 128로 맞추는 작업
  # word_to_index : 단어와 인덱스가 매핑된 딕셔너리
  # data : 리스트 형태의 원본 텍스트 데이터
  # max_seq_len : 시퀀스 최대 길이
  # update_vocab : 새로운 단어를 word_to_index에 추가할지 여부를 설정
    # True이면 새로운 단어를 사전에 추가, False이면 사전에 없는 단어는 OOV 토큰(1)으로 처리
    # (훈련 데이터에 대해 True, 테스트 데이터에 대해 False 사용)
def text_embedding_with_pad(word_to_index, data, max_seq_len, update_vocab=True):

    word_emb_table = [] # 각 텍스트의 정수 인덱스 배열을 저장

    for d in data:
        # 텍스트 정제 수행
        d = text_cleaning(d)

        # 결측값(None) 처리
        if d is None:
            d = ' '
              # continue

        # 소문자 변환 및 공백 제거
        d = d.lower().strip()

        # 마침표, 쉼표 등의 구두점 앞뒤로 띄어쓰기 추가 -> 별도의 토큰으로 분리
        d = d.replace(".", " . ").replace(",", " , ").replace("\''", " \' ").replace("\"", " \" ")

        # 정수 인덱스 리스트 생성, 0으로 초기화 (패딩(Padding) 인덱스 역할)
        word_emb = [0]*max_seq_len

        # 띄어쓰기 기준으로 문장을 단어(토큰)로 분리하여 for 문 수행해 단어와 그 위치(w_num)를 가져옴
        for w_num, w in enumerate(d.split()):
            # 문장 길이가 max_seq_len을 초과하면 중단
            if w_num == max_seq_len:
                break

            # 사전에 없는 단어 OOV(Out-Of-Vocabulary) 처리
            if w not in word_to_index:
                if update_vocab:
                    # 새로운 단어를 사전에 추가
                    word_to_index[w] = len(word_to_index)
                    word_emb[w_num] = word_to_index[w]
                else:
                    # update_vocab=False이면 모든 미등록 단어를 인덱스 1(OOV 인덱스)로 처리
                    word_emb[w_num] = 1
            else:
                # 사전에 있는 단어는 해당 정수 인덱스로 변환
                word_emb[w_num] = word_to_index[w]

        word_emb_table.append(word_emb)

    return word_to_index, np.array(word_emb_table)

    # word_emb_table : 최종 임베딩 테이블
      # 모든 문장이 정수 인덱스로 변환되고, 128 길이로 통일된 NumPy 배열

In [10]:
# 데이터셋을 훈련(Training), 검증(Validation), 테스트(Test) 세트로 분리하고,
# 사전에 구축된 word_to_index 딕셔너리를 사용하여 텍스트 데이터를 정수 인덱스로 변환하는 데이터 준비 과정

# 난수 생성을 위한 모듈
import random
# 진행 상태 표시를 위한 모듈
from tqdm import tqdm

# 훈련/검증/테스트 데이터의 인덱스 리스트 초기화
train_word_index, valid_word_index, test_word_index = [], [], []

# 전체 훈련 데이터('document')의 인덱스 리스트
random_index = list(range(len(dataset['train']['document'])))

# 난수 생성기의 시드(seed)를 42로 고정 (항상 동일한 결과가 나오도록 함)
random.seed(42)
# 인덱스 섞기 (데이터 무작위 분할을 위함)
random.shuffle(random_index)

# 학습용: 전체의 80%, 검증용: 나머지 20%
train_data_index = np.array(random_index[:int(len(random_index)*0.8)])
valid_data_index = np.array(random_index[int(len(random_index)*0.8):])

# pbar = tqdm(total = len(dataset['train']['document']))
# for i, d in enumerate(dataset['train']['document']):

# 단어 딕셔너리 초기화
word_to_index = {}
word_to_index['<PAD>'] = 0    # 패딩(Padding) 토큰 '<PAD>'에 인덱스 0 할당
word_to_index['<OOV>'] = 1    # OOV(Out-Of-Vocabulary) 토큰 '<OOV>'에 인덱스 1을 할당

# 'train' 데이터셋을 사용하여 사전을 구축하고(update_vocab=True가 default), 텍스트를 정수 시퀀스로 변환
word_to_index, train_word_index = text_embedding_with_pad(
    word_to_index, dataset['train']['document'], max_seq_len
)

# 훈련/검증 인덱스 기준으로 분리
valid_word_index = train_word_index[valid_data_index]
train_word_index = train_word_index[train_data_index]

# 라벨(label)도 동일한 인덱스로 분할
valid_label = np.array(dataset['train']['label'])[valid_data_index]
train_label = np.array(dataset['train']['label'])[train_data_index]

# 테스트 데이터셋을 정수 시퀀스로 변환 (기존에 구축된 사전은 업데이트하지 않음(update_vocab=False))
_, test_word_index = text_embedding_with_pad(word_to_index, dataset['test']['document'], max_seq_len, update_vocab=False)

In [11]:
print(dataset['train']['document'][0])
print(train_word_index[0])
print(len(word_to_index))

아 더빙.. 진짜 짜증나네요 목소리
[  202 16934  8233     4     4     4     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
314876


In [12]:
print(test_word_index[:10])

[[23122   362     0 ...     0     0     0]
 [    1     0     0 ...     0     0     0]
 [ 5586   257 56426 ...     0     0     0]
 ...
 [ 2199     1     1 ...     0     0     0]
 [  142     1 28619 ...     0     0     0]
 [15896     1 49588 ...     0     0     0]]


In [13]:
# PyTorch에서 딥러닝 모델에 데이터를 공급하는 표준 방식인 Dataset 클래스를 정의하고,
# 전처리된 데이터를 이 형식으로 변환하는 과정

# PyTorch의 데이터 처리 유틸리티인 Dataset과 DataLoader / NumPy 배열 처리 / PyTorch의 주요 라이브러리
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch

# 딥러닝 모델 입력을 위해 사용자 정의 데이터셋 클래스를 정의
  # 텍스트 데이터를 텐서로 변환하여 학습에 사용할 수 있는 torch.utils.data.Dataset 클래스를 상속
class CustomDataset(Dataset):
    # 초기화될 때 호출되는 생성자 함수
    def __init__(self, x, y):
        self.x = np.array(x) # 입력 데이터(텍스트 인덱스 시퀀스)
        self.y = np.array(y) # 레이블(긍정/부정) 데이터
        self.len = len(x)

        # 예외 처리: 입력 데이터를 NumPy 배열 -> PyTorch 텐서(Tensor) 변환
        try:
            self.x_data = torch.from_numpy(self.x).long()
        except TypeError:
            print(self.x)

        # 레이블 데이터도 정수형 PyTorch 텐서로 변환
        self.y_data = torch.from_numpy(self.y).long()

    # 특정 인덱스에 해당하는 데이터를 반환하는 필수 메서드
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

# CustomDataset을 이용해 훈련, 검증, 테스트 데이터셋을 구성하여 DataLoader로 사용할 준비
train_dataset = CustomDataset(train_word_index, train_label)
valid_dataset = CustomDataset(valid_word_index, valid_label)
test_dataset = CustomDataset(test_word_index, dataset['test']['label'])

# Training and Inference

## To be implemented

In [14]:
# PyTorch의 주요 모듈
import torch
import torch.nn as nn
import torch.optim as optim

# 데이터 로딩을 위한 DataLoader
from torch.utils.data import DataLoader

In [15]:
# Pretrained GloVe 임베딩을 모델의 Embedding Layer에 초기화하는 함수
def initialize_embedding_layer(word_emb, vocab_size, embed_dim):
    # (단어 수, 임베딩 차원) 크기의 텐서를 0으로 초기화
    embedding_weights = torch.zeros((vocab_size, embed_dim))
    for word, idx in word_to_index.items():
        # 단어가 GloVe 임베딩 사전에 존재하면, 해당 단어의 GloVe 벡터를 가중치 텐서에 복사
        if word in word_emb:
            embedding_weights[idx] = torch.tensor(word_emb[word])
        # 단어가 GloVe 사전에 없으면 (OOV 토큰 포함), 작은 랜덤 값으로 초기화
        else:
            embedding_weights[idx] = torch.randn(embed_dim) * 0.01
    return embedding_weights

In [16]:
# CNN 모델 (TextCNN의 단순화된 버전)
class ModelCNN(nn.Module):
    # 모델 초기화: 단어장 크기, 임베딩 차원, 사전 학습된 임베딩 가중치
    def __init__(self, vocab_size, embed_dim, pretrained_embeddings):
        super(ModelCNN, self).__init__()
        # Embedding Layer: 사전 학습된 가중치로 초기화하고, freeze=False로 설정하여 학습 중 가중치가 업데이트 되도록 함
        self.embedding = nn.Embedding.from_pretrained(pretrained_embeddings, freeze=False)
        # 1D 합성곱 레이어: 임베딩 차원(입력 채널), 128개 필터(출력 채널), 커널 크기 3, 패딩 1 설정
        self.conv = nn.Conv1d(embed_dim, 128, kernel_size=3, padding=1)
        # 활성화 함수 ReLU
        self.relu = nn.ReLU()
        # Max Pooling 레이어: 커널 크기 2로 설정 (길이를 절반으로 줄임)
        self.pool = nn.MaxPool1d(kernel_size=2)
        # Dropout 레이어: 과적합 방지를 위해 50%의 뉴런을 무작위로 비활성화
        self.dropout = nn.Dropout(0.5)
        # 완전 연결 레이어 (Linear): 입력 크기는 풀링 후의 결과 크기, 출력 크기는 2 (긍정/부정)
        self.fc = nn.Linear(128 * (max_seq_len // 2), 2)

    # 데이터의 순전파(Forward Propagation)를 정의
    def forward(self, x):
        # 입력 텐서(x)를 임베딩하고, Conv1D 입력 형태 (Batch, Channels, Length)에 맞게 차원을 변경
        x = self.embedding(x).permute(0, 2, 1)
        # 합성곱 -> ReLU 활성화 -> Max Pooling 적용
        x = self.pool(self.relu(self.conv(x)))
        x = self.dropout(x)
        # 텐서를 펼쳐(Flatten) FC 레이어 입력에 맞춤
        x = x.view(x.size(0), -1)
        return self.fc(x)

In [17]:
# 모델 초기화, 손실 함수 및 옵티마이저 정의

vocab_size = len(word_to_index)
# 임베딩 차원을 300으로 설정 (GloVe 300D 기준)
embed_dim = 300
# 사전 학습된 임베딩 가중치 준비
pretrained_embeddings = initialize_embedding_layer(word_emb, vocab_size, embed_dim)
# CNN 모델을 생성하고 GPU로 보냄
model = ModelCNN(vocab_size, embed_dim, pretrained_embeddings).cuda()
# 손실 함수로 Cross Entropy Loss를 사용
criterion = nn.CrossEntropyLoss()
# 최적화 도구로 Adam을 사용하고, 낮은 학습률과 L2 정규화(weight_decay)를 적용
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-5)

In [18]:
# 훈련 함수 정의
def train(model, train_loader, criterion, optimizer):
    # 모델 훈련 모드 설정
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        # 옵티마이저의 기울기를 0으로 초기화
        optimizer.zero_grad()
        # 순전파 수행 -> 예측값을 얻음
        outputs = model(inputs)
        # 손실 계산
        loss = criterion(outputs, labels)
        # 역전파 수행 -> 기울기를 계산
        loss.backward()
        # 옵티마이저를 통해 가중치 업데이트
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(train_loader)

In [19]:
# 평가 함수 정의
def evaluate(model, valid_loader, criterion):
    # 모델 평가 모드 설정
    model.eval()
    valid_loss = 0.0
    # 기울기 계산을 비활성화 (메모리 절약 및 속도 향상)
    with torch.no_grad():
        for inputs, labels in valid_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            valid_loss += loss.item()
    return valid_loss / len(valid_loader)

In [20]:
# 데이터로더 정의
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)

In [22]:
# 학습 및 검증 루프 (조기 종료 기능 추가)

# 총 epoch 수 100회 설정 / 5회 연속 개선 없을 시 조기 종료(Early Stopping)
num_epochs = 100
patience = 5
best_loss = float('inf')
patience_counter = 0

# 학습 루프 실행
for epoch in range(num_epochs):
    # 모델 훈련 모드 설정
    model.train()
    train_loss = 0.0
    for inputs, labels in DataLoader(train_dataset, batch_size=32, shuffle=True):
        optimizer.zero_grad()                     # GPU로 데이터를 이동시킴
        outputs = model(inputs.cuda())            # 순전파 및 GPU 데이터 이동
        loss = criterion(outputs, labels.cuda())  # 손실 계산 및 GPU 데이터 이동
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    # train_loss를 전체 데이터셋 크기로 정규화
    train_loss /= len(train_dataset)

    # 모델 평가 모드 설정
    model.eval()
    valid_loss = 0.0
    # 기울기 계산 비활성화
    with torch.no_grad():
        for inputs, labels in DataLoader(valid_dataset, batch_size=32, shuffle=False):
            outputs = model(inputs.cuda())            # 순전파 및 GPU 데이터 이동
            loss = criterion(outputs, labels.cuda())  # 손실 계산 및 GPU 데이터 이동
            valid_loss += loss.item()
    # valid_loss를 전체 데이터셋 크기로 정규화
    valid_loss /= len(valid_dataset)

    print(f"Train Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}")
    print(f"Eval Epoch [{epoch+1}/{num_epochs}], Loss: {valid_loss:.4f}")

    # 최적의 성능 갱신 시 모델 저장 및 조기 종료 검토
    if valid_loss < best_loss:
        best_loss = valid_loss
        patience_counter = 0
        torch.save(model.state_dict(), 'best_model.ckpt')
        print("*** Best ***")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping activated.")
            break

Train Epoch [1/100], Loss: 0.0080
Eval Epoch [1/100], Loss: 0.0131
*** Best ***
Train Epoch [2/100], Loss: 0.0033
Eval Epoch [2/100], Loss: 0.0158
Train Epoch [3/100], Loss: 0.0023
Eval Epoch [3/100], Loss: 0.0182
Train Epoch [4/100], Loss: 0.0026
Eval Epoch [4/100], Loss: 0.0184
Train Epoch [5/100], Loss: 0.0023
Eval Epoch [5/100], Loss: 0.0185
Train Epoch [6/100], Loss: 0.0019
Eval Epoch [6/100], Loss: 0.0196
Early stopping activated.


In [23]:
# Test 성능 평가

# 최적 모델 가중치를 로드
model.load_state_dict(torch.load('best_model.ckpt'))
# 모델 평가 모드 설정
model.eval()
correct, total = 0, 0
# 기울기 계산 비활성화
with torch.no_grad():
    for inputs, labels in DataLoader(test_dataset, batch_size=32, shuffle=False):
        outputs = model(inputs.cuda())
        # 가장 높은 확률을 가진 클래스를 예측값으로 선택
        _, predicted = torch.max(outputs, 1)
        # 전체 샘플 수를 누적
        total += labels.size(0)
        # 정확히 예측된 샘플 수를 누적
        correct += (predicted == labels.cuda()).sum().item()

# 최종 테스트 정확도를 계산하고 출력
print(f'Test Accuracy of the model on the test text: {100 * correct / total:.3f} %')

Test Accuracy of the model on the test text: 81.150 %
